In [150]:
import json
from pathlib import Path
from pprint import pprint, pformat

import pandas as pd
import janitor
import requests
from pydash import py_

from funcs import utils, paths
from funcs.data_processing.stage1_processing import get_ebi_data

In [2]:
ES_URL = "http://ieu-mrbssd1.epi.bris.ac.uk:26550"

r = requests.get(ES_URL)
assert r.ok
print(r.json())

{'name': '53d4a935d550', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'cfbqZlymRuas-lywIWKaOg', 'version': {'number': '7.5.1', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '3ae9ac9a93c95bd0cdc054951cf95d88e1e18d96', 'build_date': '2019-12-16T22:57:37.835892Z', 'build_snapshot': False, 'lucene_version': '8.3.0', 'minimum_wire_compatibility_version': '6.8.0', 'minimum_index_compatibility_version': '6.0.0-beta1'}, 'tagline': 'You Know, for Search'}


In [3]:
ebi_data = get_ebi_data(verbose=False)
print(ebi_data.info())
SAMPLE_SIZE = 200
ebi_sample = ebi_data[ebi_data["MAPPING_TYPE"] == "Exact"] \
    [["query", "MAPPED_TERM_LABEL", "MAPPED_TERM_URI"]] \
    .head(SAMPLE_SIZE).drop_duplicates().reset_index(drop=True)
ebi_sample

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1191 entries, 0 to 1190
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   query              1191 non-null   object
 1   MAPPED_TERM_LABEL  1191 non-null   object
 2   MAPPED_TERM_URI    1191 non-null   object
 3   MAPPING_TYPE       1191 non-null   object
 4   id                 1191 non-null   object
 5   full_id            1191 non-null   object
 6   mapping_id         1191 non-null   int64 
dtypes: int64(1), object(6)
memory usage: 65.3+ KB
None


,query,MAPPED_TERM_LABEL,MAPPED_TERM_URI
0,alzheimer s disease,Alzheimer's disease,EFO_0000249
1,endocarditis valve unspecified,endocarditis,EFO_0000465
2,family history of other conditions,family history,EFO_0000493
3,unspecified maternal hypertension,preeclampsia,EFO_0000668
4,stroke not specified as haemorrhage or infarction,stroke,EFO_0000712
...,...,...,...
195,standing height,body height,EFO_0004339
196,morning/evening person (chronotype),chronotype measurement,EFO_0008328
197,sleeplessness / insomnia,insomnia,EFO_0004698
198,snoring,snoring measurement,EFO_0008341


In [151]:
json.dumps({"index": {}})

'{"index": {}}'

In [152]:
?requests.post

Signature: requests.post(url, data=None, json=None, **kwargs)
Docstring:
Sends a POST request.

:param url: URL for the new :class:`Request` object.
:param data: (optional) Dictionary, list of tuples, bytes, or file-like
    object to send in the body of the :class:`Request`.
:param json: (optional) json data to send in the body of the :class:`Request`.
:param \*\*kwargs: Optional arguments that ``request`` takes.
:return: :class:`Response <Response>` object
:rtype: requests.Response
File:      ~/miniconda3/envs/vectology-stage2/lib/python3.8/site-packages/requests/api.py
Type:      function


In [202]:
import requests
import json

endpoint = ES_URL + '/_bulk'


#                  vvvvvv
data = [{"index": {"_index": INDEX_NAME}},
        {"tag": {"input": ['tag'], "weight":10}}]


#         vvv                                              vvv
payload = '\n'.join([json.dumps(line) for line in data]) + '\n'

r = requests.put(endpoint,
                 # `data` instead of `json`!
                 data=payload,
                 headers={           
                     # it's a requirement
                     'Content-Type': 'application/x-ndjson'
                 })

print(r.json())

{'took': 10, 'errors': False, 'items': [{'index': {'_index': 'test-efo', '_type': '_doc', '_id': 't1IdjIEBh99XIFUoQp-r', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 2, '_primary_term': 1, 'status': 201}}]}


In [215]:
def index_chunk(index_name, docs):
    index_url = ES_URL + "/" + index_name + "/_bulk"
    headers = {"Content-Type": "application/x-ndjson", "charset": "UTF-8"} 
    delim = {"index": {}}
    arr = []
    for _ in docs:
        arr.append(delim)
        arr.append(_)
    payload = "\n".join([json.dumps(_) for _ in arr]) + "\n"
    r = requests.post(index_url, data=payload, headers=headers)
    try:
        r.raise_for_status()
    except:
        print(r.text)
        print(payload)
    return True


def bulk_index(index_name, docs, chunksize=50, log_step=2):
    docs_chunks = py_.chunk(docs, size=chunksize)
    print(f"split {len(docs)} docs into {len(docs_chunks)} chunks")
    for idx, chunk in enumerate(docs_chunks):
        if idx % log_step == 0:
            print(f"# chunk {idx}")
        index_chunk(index_name, chunk)

In [155]:
INDEX_NAME = "test-efo"
index_url = ES_URL + "/" + INDEX_NAME

put_data = {
    "settings": {
        "analysis": {
            "analyzer": {
                "substring": {
                    "type": "custom",
                    "tokenizer": "standard",
                    "filter": [
                        "kstem",
                        "lowercase",
                        "substring",
                    ]
                },
                "exact": {
                    "type": "custom",
                    "tokenizer": "keyword",
                    "filter": [
                        "kstem",
                        "lowercase",
                    ]
                }
            },
            "filter": {
                "substring": {
                    "type":"shingle",
                    "output_unigrams" : True
                }
            }
        }
    },
    "mappings": {
        "properties": {
            "ent_id": {
              "type": "keyword",
            },
            "ent_term": {
                "type": "text",
            },
            "ent_term_norm": {
                "type": "text",
                "analyzer": "exact",
                "search_analyzer": "substring",
            }
        }
    }
}
pprint(put_data)

{'mappings': {'properties': {'ent_id': {'type': 'keyword'},
                             'ent_term': {'type': 'text'},
                             'ent_term_norm': {'analyzer': 'exact',
                                               'search_analyzer': 'substring',
                                               'type': 'text'}}},
 'settings': {'analysis': {'analyzer': {'exact': {'filter': ['kstem',
                                                             'lowercase'],
                                                  'tokenizer': 'keyword',
                                                  'type': 'custom'},
                                        'substring': {'filter': ['kstem',
                                                                 'lowercase',
                                                                 'substring'],
                                                      'tokenizer': 'standard',
                                                      'type': 'custom'

In [216]:
r = requests.get(index_url)
if r.ok:
    print("index exists, remove it")
    r = requests.delete(index_url)
    r.raise_for_status()
r = requests.put(index_url, json=put_data)
try:
    r.raise_for_status()
except:
    print(r.text)

index exists, remove it


In [157]:
efo_docs = [
    {
        "ent_id": _["MAPPED_TERM_URI"],
        "ent_term": _["MAPPED_TERM_LABEL"],
        "ent_term_norm": _["MAPPED_TERM_LABEL"],
    }
    for _ in ebi_sample.to_dict(orient="records")
]

additional_docs_raw = [
    {'ent_id': 'Disease_MESH_D010243', 'ent_term': 'palsy'}, 
    {'ent_id': 'Disease_MESH_D020330', 'ent_term': "bell's palsy"}, 
    {'ent_id': 'Disease_MESH_D005155', 'ent_term': 'facial nerve'}, 
    {'ent_id': 'Disease_MESH_D005148', 'ent_term': 'facial'}, 
    {'ent_id': 'Chemical_-', 'ent_term': "s'-dimethylthiobutanoylhydroxysuccinimide ester"}, 
    {'ent_id': 'Chemical_MESH_C027793', 'ent_term': "bell's palsy"}, 
    {'ent_id': 'Chemical_MESH_D013455', 'ent_term': "s'-"}, 
    {'ent_id': 'Disease_MESH_D001714', 'ent_term': "bell's mania"}, 
    {'ent_id': 'Disease_MESH_D005879', 'ent_term': "s'agit de lesions osteolytiques allant de la"}, 
    {'ent_id': 'Disease_MESH_D008945', 'ent_term': "s' mitral valve"}
    
]
additional_docs = [
    {
        "ent_id": _["ent_id"],
        "ent_term": _["ent_term"],
        "ent_term_norm": _["ent_term"],
    }
    for _ in additional_docs_raw
]

docs = efo_docs + additional_docs
# docs = additional_docs

print(len(docs))

210


In [217]:
bulk_index(index_name=INDEX_NAME, docs=docs)
# for _ in docs:
#     url = ES_URL + f"/{INDEX_NAME}" + "/_doc/"
#     r = requests.post(url, json=_)

split 210 docs into 5 chunks
# chunk 0
# chunk 2
# chunk 4


In [ ]:
url = ES_URL + f"/{INDEX_NAME}" + "/_search"

# search_data = {
#     "query": {
#         "term": {
#             "ent_id": {
#                 # "value": "EFO_0007878"
#                 # "value": "EFO_0007878"
#                 # "value": "EFO_0007878",
#                 # "value": "EFO_0009552",
#                 # "value": "Disease_MESH_D010243"
#                 # "value": "Disease_MESH_D005879"
#                 # "value": "Disease_MESH_D020330"
#                 "value": "Chemical_MESH_C027793"
#                 
#             }
#         }
#     }
# }

search_data = {
    "query": {
        "match": {
            "ent_term_norm": "facial"
        }
    }
}

# search_data = {
#     "size": 10,
#     "sort": {"timestamp": "desc"},
#     "query": {"match_all": {}}
# }

# search_data = None

r = requests.get(url, json=search_data)
try:
    r.raise_for_status()
except:
    print(r.text)
finally:
    pprint(r.json())

{'_shards': {'failed': 0, 'skipped': 0, 'successful': 1, 'total': 1},
 'hits': {'hits': [{'_id': 'iFLbi4EBh99XIFUoP5rh',
                    '_index': 'test-efo',
                    '_score': 4.946393,
                    '_source': {'ent_id': 'Disease_MESH_D005148',
                                'ent_term': 'facial',
                                'ent_term_norm': 'facial'},
                    '_type': '_doc'}],
          'max_score': 4.946393,
          'total': {'relation': 'eq', 'value': 1}},
 'timed_out': False,
 'took': 0}


In [138]:
def search_term(term: str):
    search_data = {
        # "query": {
        #     "match": {
        #         "ent_term_norm": term
        #     }
        # }
        "query": {
            "bool": {
                "should": {
                    "match": {
                        "ent_term_norm": {
                            "query": term
                        }
                    }
                }
            }
        }
    }

    url = ES_URL + f"/{INDEX_NAME}" + "/_search"
    r = requests.get(url, json=search_data)
    try:
        r.raise_for_status()
    except:
        print(r.text)
    res =  r.json()["hits"]["hits"]
    return res

In [219]:
# QUERY = "family history"
# QUERY = "acute hepatitis"
QUERIES = [
    "Alzheimer's disease",
    "alzheimer s disease",
    "alzheimer disease",
    "family history",
    "acute hepatitis",
    "liver disease",
    "I am a composite term with both one term family history and another term stroke in the label",
    "bell's palsy/facial nerve palsy",
    "bell's palsy",
    "bell palsy",
    "facial nerve",
    "facial",
]

for idx, query in enumerate(QUERIES):
    print(f"#{idx} {query}")
    search_res = search_term(query)
    pprint(search_res)
    print("\n")

#0 Alzheimer's disease
[{'_id': 'olIhjIEBh99XIFUo36CP',
  '_index': 'test-efo',
  '_score': 4.946393,
  '_source': {'ent_id': 'EFO_0000249',
              'ent_term': "Alzheimer's disease",
              'ent_term_norm': "Alzheimer's disease"},
  '_type': '_doc'}]


#1 alzheimer s disease
[]


#2 alzheimer disease
[]


#3 family history
[{'_id': 'pFIhjIEBh99XIFUo36CP',
  '_index': 'test-efo',
  '_score': 4.946393,
  '_source': {'ent_id': 'EFO_0000493',
              'ent_term': 'family history',
              'ent_term_norm': 'family history'},
  '_type': '_doc'}]


#4 acute hepatitis
[]


#5 liver disease
[{'_id': 'qlIhjIEBh99XIFUo36CP',
  '_index': 'test-efo',
  '_score': 4.946393,
  '_source': {'ent_id': 'EFO_0001421',
              'ent_term': 'liver disease',
              'ent_term_norm': 'liver disease'},
  '_type': '_doc'}]


#6 I am a composite term with both one term family history and another term stroke in the label
[{'_id': 'pFIhjIEBh99XIFUo36CP',
  '_index': 'test-efo',
 

In [149]:
output_file = Path(".") / "es_ebi_sample_search_res.txt"

with output_file.open("w") as f:
    for idx, _ in enumerate(ebi_sample.to_dict(orient="records")):
        print(idx)
        query = _["query"]
        expected = _["MAPPED_TERM_LABEL"]
        line = f"#{idx}\n  query: {query}\n  expected: {expected}"
        f.write(line + "\n")
        search_res = search_term(query)
        search_res_pformat = pformat(search_res)
        f.write(search_res_pformat + "\n")
        f.write("\n--------\n")

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
